In [14]:
import pandas as pd
import ast
from collections import Counter
import re
from sklearn.model_selection import train_test_split
import ast

import json
import spacy
nlp = spacy.load("en_core_web_sm")
from tqdm import tqdm
from collections import defaultdict 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from collections import Counter
import os
from os import walk
import json
from tqdm import tqdm

In [20]:
my_path = './Recipe_dataset/RecipeFullDataset/'
filenames = list(set(next(walk(my_path), (None,None,[]))[2]))
print(len(filenames))

501


In [22]:
def text_cleaning(ingredient_list):

    clean_list = []
    for i in range(len(ingredient_list)):
        recipe_doc = nlp(str(ingredient_list[i]))
        temp_list = []
        for token in recipe_doc:
            temp_list.append(token.lemma_.lower())
        if len(temp_list) > 0:
            clean_text = ' '.join(temp_list)
            clean_list.append(clean_text)
    return clean_list

In [24]:
def get_info(index_list, recipe_dict, country_ref, index_name='Train_Variations'):
    
    # KB_rawtext_avg, train_rawtext_avg, valid_rawtext_avg, test_rawtext_avg = [], [] ,[] ,[]
    # KB_cleantext_avg, train_cleantext_avg, valid_cleantext_avg, test_cleantext_avg = [], [] ,[] ,[]
    # KB_uniq_words, train_uniq_words, valid_uniq_words, test_uniq_words = [],[],[],[]
    # KB_clean_words, train_clean_words, valid_clean_words, test_clean_words = [], [], [], [] 
    
    country_list, ingr_list, same_country_ingr, diff_country_ingr = [], [], [], []
    recipe_raw_len, recipe_clean_len, uniq_raw_words, uniq_clean_words = [], [], [], []
    
    for index in index_list:
        temp_dict = recipe_dict[index_name][index]
        country = temp_dict['country']
        country_list.append(country)
        ingr_list.extend(ast.literal_eval(recipe_dict[index_name][index]['ingredient_list']))
        
        raw_recipe = ' '.join(recipe_dict[index_name][index]['recipe_raw'])
        uniq_raw_words.append(len(set(raw_recipe.split())))
        recipe_raw_len.append(len(raw_recipe.split()))
        recipe_clean_len.append(len(recipe_dict[index_name][index]['recipe_clean'].split()))  ### type string
        uniq_clean_words.append(len(set(recipe_dict[index_name][index]['recipe_clean'].split())))
        
        if country == country_ref:
            same_country_ingr.extend(ast.literal_eval(recipe_dict[index_name][index]['ingredient_list']))
        else:
            diff_country_ingr.extend(ast.literal_eval(recipe_dict[index_name][index]['ingredient_list']))
            
    
    #List of unique ingredients in the variation
    uniq_ingr_list = list(set(text_cleaning(ingr_list)))
    uniq_same_ingr_list = list(set(text_cleaning(same_country_ingr)))
    uniq_diff_ingr_list = list(set(text_cleaning(diff_country_ingr)))
    avg_raw_len, avg_clean_len, avg_raw_uniq, avg_clean_uniq = 0,0,0,0
    if len(recipe_raw_len) > 0:
        avg_raw_len = sum(recipe_raw_len) / len(recipe_raw_len)
    if len(recipe_clean_len) > 0:
        avg_clean_len = sum(recipe_clean_len) / len(recipe_clean_len)
    if len(uniq_raw_words) > 0:
        avg_raw_uniq = sum(uniq_raw_words) / len(uniq_raw_words)
    if len(uniq_clean_words) > 0:
        avg_clean_uniq = sum(uniq_clean_words) / len(uniq_clean_words)
    
    return country_list, uniq_ingr_list, uniq_same_ingr_list, uniq_diff_ingr_list, avg_raw_len, avg_clean_len, avg_raw_uniq, avg_clean_uniq

In [26]:
def get_new_ingr(var_ingr_list, same_country_list, diff_country_list, ref_ingr_list):
    
    new_ingr, new_ingr_same, new_ingr_diff = 0,0,0
    for ingredient in var_ingr_list:
        if ingredient not in ref_ingr_list:
            new_ingr += 1
            if ingredient in same_country_list:
                new_ingr_same += 1
            if ingredient in diff_country_list:
                new_ingr_diff += 1
    return new_ingr, new_ingr_same, new_ingr_diff

In [28]:

#######Lists to save as csv for info per recipe##########
len_train_var, len_valid_var, len_test_var = [], [], []
len_other_country_var, len_same_country_var = [], []
len_KB = []
ratio_KB_tot_var = []

#number of recipes names used
nb_names = []

#Getting the recipe country and the most present country for variations
recipe_country, top_1_country, top_2_country, top_3_country = [],[],[],[]
train_country_div, valid_country_div, test_country_div, recipe_country_div = [],[],[],[] 
nb_same_country, nb_diff_country = [], []

KB_ingr_size, train_ingr_size, valid_ingr_size, test_ingr_size = [], [], [], []
same_country_ingr_size, diff_country_ingr_size = [], []
total_new_ingr, train_new_ingr, valid_new_ingr, test_new_ingr = [], [], [], []
same_country_new_ingr, diff_country_new_ingr = [], []

#Countring the number of unique words and the length of raw and cleaned recipes
KB_rawtext_avg, train_rawtext_avg, valid_rawtext_avg, test_rawtext_avg = [], [] ,[] ,[]
KB_cleantext_avg, train_cleantext_avg, valid_cleantext_avg, test_cleantext_avg = [], [] ,[] ,[]
KB_uniq_words, train_uniq_words, valid_uniq_words, test_uniq_words = [],[],[],[]
KB_clean_words, train_clean_words, valid_clean_words, test_clean_words = [], [], [], [] 

#Stats for the KB sizes and the compared variation sizes
for k in tqdm(range(len(filenames))):
    filename = filenames[k]
    file_path = my_path + filename
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)
    
    KB_base = recipe_dict['Reference_Base']
    KB_indexes = [item for item in KB_base.keys() if item!='AllIngredients']
    
    train_indexes = list(recipe_dict['Train_Variations'].keys())
    valid_indexes = list(recipe_dict['Valid_Variations'].keys())
    test_indexes = list(recipe_dict['Test_Variations'].keys())
    
    len_KB.append(len(KB_indexes))
    len_train_var.append(len(train_indexes))
    len_valid_var.append(len(valid_indexes))
    len_test_var.append(len(test_indexes))
    total_var = len(train_indexes) + len(valid_indexes) + len(test_indexes)
    ratio_KB_tot_var.append(len(KB_indexes)/total_var)
    
    KB_country = recipe_dict['Country']
    recipe_country.append(KB_country)
    
    #Nb of variation names used in the data collection
    nb_names.append(len(recipe_dict['Name variations'].split(' | ')))

    KB_ingr_list = recipe_dict['Reference_Base']['AllIngredients']
    KB_ingr_size.append(len(KB_ingr_list))
    
    #Countring the number of unique words and the length of raw and cleaned recipes
    temp_KB_rawtext, temp_KB_cleantext, temp_KB_uniq_words, temp_KB_clean_words = [], [], [], []
    for index in KB_indexes:
        sentences = recipe_dict['Reference_Base'][index]['recipe_raw']
        raw_recipe = ' '.join(sentences)
        temp_KB_uniq_words.append(len(set(raw_recipe.split())))
        #temp_KB_rawtext.append(sum(len(sentence.split()) for sentence in sentences))
        temp_KB_rawtext.append(len(raw_recipe.split()))
        temp_KB_cleantext.append(len(recipe_dict['Reference_Base'][index]['recipe_clean'].split()))  ### type string
        temp_KB_clean_words.append(len(set(recipe_dict['Reference_Base'][index]['recipe_clean'].split())))
    
    KBavg_raw_len = sum(temp_KB_rawtext) / len(temp_KB_rawtext)
    KBavg_clean_len = sum(temp_KB_cleantext) / len(temp_KB_cleantext)
    KBavg_raw_uniq = sum(temp_KB_uniq_words) / len(temp_KB_uniq_words)
    KBavg_clean_uniq = sum(temp_KB_clean_words) / len(temp_KB_clean_words)
    
    KB_rawtext_avg.append(KBavg_raw_len)
    KB_cleantext_avg.append(KBavg_clean_len)
    KB_uniq_words.append(KBavg_raw_uniq)
    KB_clean_words.append(KBavg_clean_uniq)

    ## Getting all statistics for train, valid, test
    #def get_info(index_list, recipe_dict, country_ref, index_name='Train_Variations'):
    country_list_train, ingr_list_train, same_train_ingr, diff_train_ingr, \
    train_raw_len, trian_clean_len, trian_raw_uniq, train_clean_uniq = get_info(train_indexes, recipe_dict, KB_country, index_name='Train_Variations')
    country_list_valid, ingr_list_valid, same_valid_ingr, diff_valid_ingr, \
    valid_raw_len, valid_clean_len, valid_raw_uniq, valid_clean_uniq = get_info(valid_indexes, recipe_dict, KB_country, index_name='Valid_Variations')
    country_list_test, ingr_list_test, same_test_ingr, diff_test_ingr, \
    test_raw_len, test_clean_len, test_raw_uniq, test_clean_uniq = get_info(test_indexes, recipe_dict, KB_country, index_name='Test_Variations')
    
    
    #Getting the recipe country and the most present country for variations
    total_pays = country_list_train + country_list_valid + country_list_test
    dict_pays = Counter(total_pays)
    # Sort the dictionary items by value in descending order and get the top 3
    top_3_keys = sorted(dict_pays, key=dict_pays.get, reverse=True)[:3]
    same_country = dict_pays[KB_country]
    diff_country = len(total_pays) - same_country
    
    #Country statistics
    country1, country2, country3 = None, None, None
    if len(top_3_keys) >= 1:
        country1 = top_3_keys[0]
    if len(top_3_keys) >= 2:
        country2 = top_3_keys[1]
    if len(top_3_keys) >= 3:
        country3 = top_3_keys[2]
    top_1_country.append(country1)
    top_2_country.append(country2)
    top_3_country.append(country3)
    nb_same_country.append(same_country)
    nb_diff_country.append(diff_country)
    recipe_country_div.append(len(dict_pays))
    train_country_div.append(len(list(set(country_list_train))))
    valid_country_div.append(len(list(set(country_list_valid))))
    test_country_div.append(len(list(set(country_list_test))))
    
    #ingredients statistics
    same_country_tot = list(set(same_train_ingr + same_valid_ingr + same_test_ingr))
    diff_country_tot = list(set(diff_train_ingr + diff_valid_ingr + diff_test_ingr))
    tot_ingr_var = list(set(ingr_list_train + ingr_list_valid + ingr_list_test))
    
    train_ingr_size.append(len(ingr_list_train))
    valid_ingr_size.append(len(ingr_list_valid))
    test_ingr_size.append(len(ingr_list_test))
    same_country_ingr_size.append(len(same_country_tot))
    diff_country_ingr_size.append(len(diff_country_tot))
    
    new_ingr, same_new_inrg, diff_new_ingr = get_new_ingr(tot_ingr_var, same_country_tot, diff_country_tot, KB_ingr_list)
    new_ingr_train, _, _ = get_new_ingr(ingr_list_train, same_train_ingr, diff_train_ingr, KB_ingr_list)
    new_ingr_valid, _, _ = get_new_ingr(ingr_list_valid, same_valid_ingr, diff_valid_ingr, KB_ingr_list)
    new_ingr_test, _, _ = get_new_ingr(ingr_list_test, same_test_ingr, diff_test_ingr, KB_ingr_list)
    
    
    total_new_ingr.append(new_ingr)
    train_new_ingr.append(new_ingr_train)
    valid_new_ingr.append(new_ingr_valid)
    test_new_ingr.append(new_ingr_test)
    same_country_new_ingr.append(same_new_inrg)
    diff_country_new_ingr.append(diff_new_ingr)
    
    ## Recipe sizes 
    train_rawtext_avg.append(train_raw_len)
    valid_rawtext_avg.append(valid_raw_len)
    test_rawtext_avg.append(test_raw_len)
    train_cleantext_avg.append(trian_clean_len)
    valid_cleantext_avg.append(valid_clean_len)
    test_cleantext_avg.append(test_clean_len)
    train_uniq_words.append(trian_raw_uniq)
    valid_uniq_words.append(valid_raw_uniq)
    test_uniq_words.append(test_raw_uniq)
    train_clean_words.append(train_clean_uniq)
    valid_clean_words.append(valid_clean_uniq)
    test_clean_words.append(test_clean_uniq)
    
    
df = pd.DataFrame({'ID_Name': filenames,
                   'Nb recipes in KB': len_KB, 'Train - Nb variations': len_train_var, 'Valid - Nb variations': len_valid_var, 'Test - Nb variations': len_test_var, 
                   'Ratio recipes KB variations': ratio_KB_tot_var, 'Ref Country': recipe_country, 'Nb different names': nb_names, 'KB - Nb ingredients': KB_ingr_size,
                   'KB - Text length':KB_rawtext_avg, 'Train Var - Text length': train_rawtext_avg, 'Valid Var - Text length': valid_rawtext_avg, 
                   'Test Var - Text length': test_rawtext_avg,
                   'KB - Clean length':KB_cleantext_avg, 'Train Var - Clean length': train_cleantext_avg, 'Valid Var - Clean length': valid_cleantext_avg, 
                   'Test Var - Clean length': test_cleantext_avg,
                   'KB - Nb Tokens':KB_uniq_words, 'Train Var - Nb Tokens': train_uniq_words, 'Valid Var - Nb Tokens': valid_uniq_words, 'Test Var - Nb Tokens': test_uniq_words,
                   'KB - Clean Nb tokens':KB_clean_words, 'Train Var - Clean Nb tokens': train_clean_words, 'Valid Var - Clean Nb tokens': valid_clean_words, 
                   'Test Var - Clean Nb tokens': test_clean_words, 'Top1 Country': top_1_country, 'Top2 Country': top_2_country, 'Top3 Country': top_3_country, 
                   'Nb recipes same country': nb_same_country, 'Nb recipes different country': nb_diff_country, 'Total Nb different countries': recipe_country_div, 
                   'Train - Nb different country': train_country_div, 'Valid - Nb different country': valid_country_div, 'Test - Nb different country' : test_country_div,
                   'Train - Nb ingredients': train_ingr_size, 'Valid - Nb ingredients': valid_ingr_size, 'Test - Nb ingredients': test_ingr_size, 
                   'Same Country - Nb ingredients': same_country_ingr_size, 'Different Country - Nb ingredients': diff_country_ingr_size, 
                   'Total - Nb new ingredients': total_new_ingr, 'Train - Nb new ingredients': train_new_ingr, 'Valid - Nb new ingredients': valid_new_ingr, 
                   'Test - Nb new ingredients': test_new_ingr, 'Same Country - Nb new ingredients': same_country_new_ingr,
                   'Different Country - Nb new ingredients': diff_country_new_ingr})

100%|██████████████████████████████████████████████████████████████████████████████| 501/501 [2:11:07<00:00, 15.70s/it]


In [29]:
df.to_csv('CulturalNovelty_DatasetStatistics.csv', index=False)